In [1]:
import os
import requests
from groq import Groq
import json
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup

In [2]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("No API Keys found")
elif api_key.strip() !=api_key:
    print("API Key might have tabs or spaces in it")
else:
    print("API Key found and looks good")

API Key found and looks good


In [3]:
MODEL = "llama-3.3-70b-versatile"
groq = Groq()

In [11]:
class Website:

    url:str
    body:str
    title:str
    links:str
    text:str

    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n",strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
temp = Website("https://www.anthropic.com")
print(temp.get_contents(), temp.links)

In [13]:
link_system_prompt = "You are provided with a list of links found on a website. \
You are able to decide which of the links would be most relevant to include in determining the possible problems/improvements that this company might need help with from a service based IT firm, based on their business or product.Refer to the examples below in determining which links would be relevant.\n"
link_system_prompt += """ 
When analyzing a company's website to identify potential problems or areas for improvement that could be addressed by a service-based IT firm, focus on these types of links or sections:

1. About Us
Why Useful: Provides insights into the company’s mission, vision, and goals. You can understand their business focus and assess whether their technology aligns with their objectives.
Potential Indicators:
Outdated or unclear branding or messaging.
Lack of digital transformation alignment.
2. Products/Services
Why Useful: Details on what the company offers to its customers. Evaluate whether their offerings are user-friendly, scalable, or in line with current trends.
Potential Indicators:
Missing features that competitors provide.
Outdated product interfaces or designs.
No clear differentiation in the market.
3. Careers
Why Useful: Highlights the company’s hiring trends and skillset requirements.
Potential Indicators:
Hiring for IT roles that indicate gaps in their internal capabilities.
A lack of positions that would be critical for modern technology needs (e.g., DevOps, cybersecurity).
4. Contact Us
Why Useful: Analyzing this can reveal inefficiencies in how the company handles inquiries.
Potential Indicators:
Lack of automation (e.g., no chatbots or CRM systems).
Manual forms with no integration into CRM platforms.
5. Case Studies/Portfolio
Why Useful: Demonstrates the company’s past projects and achievements. It can help assess how innovative or traditional their solutions are.
Potential Indicators:
A lack of recent projects, signaling stagnation.
Limited use of technology in their solutions.
6. Blogs/Newsroom
Why Useful: Provides updates about the company’s innovations and challenges.
Potential Indicators:
Rare or outdated blog posts suggesting low engagement or thought leadership.
Lack of mentions of emerging technologies.
7. Technology/Infrastructure
Why Useful: Describes the technology stack or platforms the company uses.
Potential Indicators:
Outdated tools and frameworks that need modernization.
Missing integration capabilities.
8. Investor Relations (if applicable)
Why Useful: Includes financial reports and strategic goals.
Potential Indicators:
Comments about cost overruns or inefficiencies.
Mention of plans to improve digital transformation.
9. Customer Support
Why Useful: Highlights how they interact with and support their customers.
Potential Indicators:
Limited support channels (e.g., no live chat, AI-based ticketing).
Poor response times or unresolved customer pain points.
10. Privacy Policy/Terms of Service
Why Useful: Reflects how the company handles data privacy and compliance.
Potential Indicators:
Non-compliance with GDPR, CCPA, or other regulations.
A lack of emphasis on cybersecurity.
11. E-commerce/Store (if applicable)
Why Useful: Provides insight into the user experience and checkout processes.
Potential Indicators:
Complex or unintuitive purchasing processes.
Missing payment options or modern e-commerce features.
12. Social Media Links
Why Useful: Indicates how active and engaged the company is in its market.
Potential Indicators:
Lack of responsiveness to customer feedback.
No use of social media analytics or automation.
\n\n
"""
link_system_prompt+= "You should respond in JSON as in the examples below."
link_system_prompt+= """
{
"links":[
    {"type":"about page","url":"https://full.url/goes/here/about/","area_of_improvement":"Data Services"},
    {"type":"careers page","url":"https://another.full.url/careers/","area_of_improvement":"AI Services"}
    ]
}
\n
"""
link_system_prompt += """

"links":[
    {"type":"contact", "url":"https://url.com/yadayada/contact","area_of_improvement":"Analytics Services"},
    {"type":"customers","url":"https://website.net/us/customers","area_of_improvement":"Cloud Development Services"}
]
\n
"""
link_system_prompt += """

"links":[
    {"type":"business", "url":"https://tiny.url/business","area_of_improvement":"Automation Services"},
    {"type":"articles","url":"https://mg.com/us/articles","area_of_improvement":"Web Development Services"}
]
"""

In [14]:
print(link_system_prompt)

You are provided with a list of links found on a website. You are able to decide which of the links would be most relevant to include in determining the possible problems/improvements that this company might need help with from a service based IT firm, based on their business or product.Refer to the examples below in determining which links would be relevant.

 
When analyzing a company's website to identify potential problems or areas for improvement that could be addressed by a service-based IT firm, focus on these types of links or sections:

1. About Us
Why Useful: Provides insights into the company’s mission, vision, and goals. You can understand their business focus and assess whether their technology aligns with their objectives.
Potential Indicators:
Outdated or unclear branding or messaging.
Lack of digital transformation alignment.
2. Products/Services
Why Useful: Details on what the company offers to its customers. Evaluate whether their offerings are user-friendly, scalable